This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

# Barry--Goldman Algorithm

We can try to use the algorithm described in the
[notebook about non-uniform Catmull--Rom splines](../euclidean/catmull-rom-non-uniform.ipynb)
using [Slerp](slerp.ipynb) instead of linear interpolations,
just as we have done with [De Casteljau's algorithm](de-casteljau.ipynb).

In [ ]:
def slerp(one, two, t):
    return (two * one.inverse())**t * one

In [ ]:
def barry_goldman(rotations, times, t):
    q0, q1, q2, q3 = rotations
    t0, t1, t2, t3 = times
    return slerp(
        slerp(
            slerp(q0, q1, (t - t0) / (t1 - t0)),
            slerp(q1, q2, (t - t1) / (t2 - t1)),
            (t - t0) / (t2 - t0)),
        slerp(
            slerp(q1, q2, (t - t1) / (t2 - t1)),
            slerp(q2, q3, (t - t2) / (t3 - t2)),
            (t - t1) / (t3 - t1)),
        (t - t1) / (t2 - t1))

Example:

In [ ]:
import numpy as np

[helper.py](helper.py)

In [ ]:
from helper import angles2quat, animate_rotations, display_animation

In [ ]:
q0 = angles2quat(0, 0, 0)
q1 = angles2quat(90, 0, 0)
q2 = angles2quat(90, 90, 0)
q3 = angles2quat(90, 90, 90)

In [ ]:
t0 = 0
t1 = 1
t2 = 4
t3 = 7

In [ ]:
frames = 50

In [ ]:
times = np.linspace(t1, t2, frames)

In [ ]:
rotations = [barry_goldman([q0, q1, q2, q3], [t0, t1, t2, t3], t) for t in times]

In [ ]:
ani = animate_rotations({
    'Barry–Goldman (q0, q1, q2, q3)': rotations,
    'Slerp (q1, q2)': slerp(q1, q2, np.linspace(0, 1, frames)),
}, figsize=(6, 3))

In [ ]:
display_animation(ani, default_mode='once')